In [6]:
import pdfplumber
import re
import json

pdf_path = "../Documentation/2024 DOR Property Records Agenda.pdf"

addresses = []

# Pattern for lines ending with "Town of Agenda"
footer_pattern = re.compile(r"(.+?, Town of Agenda)")

# Pattern to detect a single-line address (from "Property address:")
single_line_address = re.compile(r"^\d{5,}\s.+")  # e.g. "16617 North Rd"

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        if not text:
            continue

        lines = text.split("\n")
        
        # Check for footer-style addresses
        for line in lines:
            # Check if line contains an address in the footer
            match = footer_pattern.search(line)
            if match:
                address = match.group(1).strip()
                addresses.append(address)
        
        # Check for addresses under "Property address:"
        for idx, line in enumerate(lines):
            if "Property address:" in line:
                # Look at the next non-empty line
                for next_line in lines[idx+1:]:
                    next_line = next_line.strip()
                    if next_line:
                        if single_line_address.match(next_line):
                            print(next_line)
                            addresses.append(next_line)
                        break

# Deduplicate
addresses = list(set(addresses))

with open("addresses.json", "w") as f:
    json.dump(addresses, f, indent=2)
